# [요구사항1] titanic_dataset.py 분석

In [2]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [3]:
class TitanicDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.FloatTensor(X)
    self.y = torch.LongTensor(y)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str

- `TitanicDataset` 클래스는 파이토치의 `Dataset` 클래스를 기본적으로 상속받음
- 모델을 훈련시키는데 필요한 데이터셋 클래스
- X, y 모두 텐서로 초기화함

In [4]:
class TitanicTestDataset(Dataset):
  def __init__(self, X):
    self.X = torch.FloatTensor(X)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str


- `TitanicTestDataset` 클래스 역시 `Dataset` 클래스를 상속받음
- `TitanicDataset`이 학습 데이터셋을 정의하는 반면에 `TitanicTestDataset`은 <b>테스트</b> 데이터셋을 정의함


In [5]:
def get_preprocessed_dataset():
    # titanic_dataset.py 파일의 절대경로를 저장하여 훈련 및 테스트 데이터 경로를 획득
    CURRENT_FILE_PATH = "/Users/jaewoogwak/git/link_dl/_03_your_code/homework/hw2"
    # CURRENT_FILE_PATH = os.path.dirname("hw2.ipynb")
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    # 훈련 및 테스트 데이터 경로를 통해 pandas 라이브러리로 csv 파일을 읽어옴
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    # 훈련 데이터와 테스트 데이터를 pandas.concat으로 합쳐줌
    all_df = pd.concat([train_df, test_df], sort=False)


    # 전처리 단계 1: Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    all_df = get_preprocessed_dataset_1(all_df)

    # 전처리 단계 2: name 값 가공하기 
    all_df = get_preprocessed_dataset_2(all_df)

    # 전처리 단계 3: age 결측치 메우기
    all_df = get_preprocessed_dataset_3(all_df)

    # 전처리 단계 4: family_num(가족수) 컬럼 추가
    all_df = get_preprocessed_dataset_4(all_df)

    # 전처리 단계 5: honorific 값 개수 줄이기
    all_df = get_preprocessed_dataset_5(all_df)

    # 전처리 단계 6: 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기 (문자열을 수치값으로 변경)
    all_df = get_preprocessed_dataset_6(all_df)

    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

In [6]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    # 결측치(missing value)란 데이터에서 값이 비어있는 데이터를 말한다.
    # 결측치가 있는 데이터는 학습 과정에서 문제를 일으킬 수 있으므로 적절한 처리 과정이 필요하다.

    # Pclass별 Fare(요금) 평균 계산
    # Pclass = 1인 1등석이 Fare 평균이 가장 높음.
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()

    # 컬럼 이름을 Fare_mean으로 변경
    Fare_mean.columns = ["Pclass", "Fare_mean"]

    # 기존 데이터와 Fare_mean을 Pclass열 기준으로 테이블 조인함. 즉, 각 승객에 대한 평균 Fare값도 추가됨
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")

    # Fare 열에서 결측치를 찾아 Fare_mean 열의 값으로 대체함.
    # 실제로 all_df의 Fare 컬럼에서 1043번째 값이 NaN임
    # print(all_df["Fare"][1043]) >> NaN
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]
    return all_df

In [7]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    # 기존 name값에서 ,. 를 기준으로 분리함. 즉 승객 이름이 세 부분으로 분리됨
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)

    # name_df는 family_name, honorific, name으로 구성됨
    name_df.columns = ["family_name", "honorific", "name"]

    # 공백 제거
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()

    # 이름 값들을 가공한 뒤 기존 데이터에 붙여줌
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

In [8]:
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    # 호칭별 나이 평균값 구하기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()

    # 컬럼 이름 변경: age -> honorific_age_mean
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]

    # 기존 데이터에 honorific 열 기준으로 테이블 조인
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")

    # 결측값 메워주기
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]

    # 결측값을 메워줬으니 기존의 honorific_age_mean은 삭제해줌
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

In [9]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    # 동승한 자매/배우자의 수 + 동승한 부모/자식의 수로 가족 수 계산하여 family_num열 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    # family_num이 0이면 alone은 1
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    # 결측치가 있는 경우 0으로 채움
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

In [10]:
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    # Mr, Miss, Mrs, Master를 제외한 호칭은 others로 바꿈
    # honorific 값 개수를 줄여줌으로써 호칭이 너무 다양할 경우 학습이 제대로 되지 않는 것을 방지
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    # Embarked열 값의 결측치를 missing으로 바꿔줌
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df

In [11]:
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    # category_features에 저장되는 컬럼들은 주로 문자열을 담고 있음.
    category_features = all_df.columns[all_df.dtypes == "object"] # >> Index(['Sex', 'Embarked', 'honorific'], dtype='object')
    
    # 문자열 값을 수치화 해줌
    # ex) 아래 루프를 지난 뒤 print(all_df["Sex"][0:10])를 실행해보면 성별 값이 0 or 1로 바뀐 것을 확인할 수 있음
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        # LabelEncoder 객체 생성
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
          le = le.fit(all_df[category_feature])
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

In [12]:
from torch import nn

In [13]:
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    # 활성화 함수 Linear, ReLU, Linear, ReLU, Linear를 받아서 순서대로 모듈 실행
    self.model = nn.Sequential(
      nn.Linear(n_input, 30),
      nn.ReLU(),
      nn.Linear(30, 30),
      nn.ReLU(),
      nn.Linear(30, n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

In [14]:
def test(test_data_loader):
  print("[TEST]")
  # test data loader로부터 batch를 차례차례 가져옴
  batch = next(iter(test_data_loader))
  print("{0}".format(batch['input'].shape))
  # input=11, output=2인 모델 생성
  my_model = MyModel(n_input=11, n_output=2)
  # 모델에 input을 넣어 output을 얻음 output_batch.shape = (418,2)
  output_batch = my_model(batch['input'])

  # prediction_batch는 예측값
  # output_batch에서 가장 높은 값의 인덱스를 찾음 (레이블 역할)
  prediction_batch = torch.argmax(output_batch, dim=1)
  for idx, prediction in enumerate(prediction_batch, start=892):
      print(idx, prediction.item())

In [15]:
if __name__ == "__main__":
    # 8:2로 나눈 train_dataset, validation_dataset을 준비 (Dataloader에 넣기 위함)
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

  print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))
  print("#" * 50, 1)

  for idx, sample in enumerate(train_dataset):
    print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

  print("#" * 50, 2)

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    # 훈련 데이터
  print("[TRAIN]")
  for idx, batch in enumerate(train_data_loader):
      # my_model = MyModel(n_input=11, n_output=2) 이기 때문에
      # Data의 input size는 11이고 batch 사이즈는 16 -> (16,11)
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    # 검증 데이터
  print("[VALIDATION]")
  for idx, batch in enumerate(validation_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("#" * 50, 3)

  test(test_data_loader)

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 3.0000,  1.0000, 16.0000,  0.0000,  0.0000,  8.0500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 1
1 - tensor([ 3.0000,  1.0000, 34.0000,  0.0000,  0.0000,  6.4958,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
2 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  8.0500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
3 - tensor([ 1.0000,  1.0000, 54.0000,  0.0000,  1.0000, 77.2875,  2.0000, 87.5090,
         2.0000,  1.0000,  0.0000]): 0
4 - tensor([ 3.0000,  0.0000, 22.0000,  0.0000,  0.0000,  7.7500,  1.0000, 13.3029,
         1.0000,  0.0000,  1.0000]): 0
5 - tensor([ 3.0000,  0.0000, 22.0000,  8.0000,  2.0000, 69.5500,  2.0000, 13.3029,
         1.0000, 10.0000,  0.0000]): 0
6 - tensor([ 3.0000,  1.0000, 30.0000,  1.0000,  0.0000, 16.1000,  2.0000, 13.3029,
         2.0000,  1.0000,  0.0000]): 0
7 - tensor([ 1.00

# [요구사항2] titanic 딥러닝 모델 훈련 코드 및 Activation Function 변경해보기

In [16]:
import os

from pathlib import Path


import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader, Dataset
from datetime import datetime
import wandb
import argparse


import sys

BASE_PATH = str(Path(os.path.abspath('hw2.ipynb')).resolve().parent.parent.parent.parent)


import sys
sys.path.append(BASE_PATH)

print("BASE PATH", BASE_PATH)

from _03_your_code.homework.hw2.titanic_dataset import get_preprocessed_dataset



# Train, Validation dataset을 반환하는 함수
def get_data():
  # titanic_dataset.py에서 사용했던 get_preprocessed_dataset 함수를 통해 전처리된 데이터 얻기
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
  # 해당 데이터를 Dataloader에 넣어 훈련에 사용할 수 있는 dataloader 얻기
  # batch size는 wandb.default = 512임.
  # validation dataset size = validation data loader's batch size = 178로 validation dataset을 한번에 훈련시킴
  train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))

  return train_data_loader, validation_data_loader


class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    print("wandb.confi.hidden unit list", wandb.config.n_hidden_unit_list)

    # nn.Sequential은 인자로 들어온 모듈들에 입력값을 순서대로 적용해준다.
    # Input layer - hidden layer(1) - Output layer 구조로 이루어짐
    # (input, 20) (20, 20) (20, output)
    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      nn.ELU(),
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      nn.ELU(),
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x


# Model & Optimizer를 얻는 함수
def get_model_and_optimizer():
  # titanic dataset은 feature가 11개이고 output(생존여부)이 1
  my_model = MyModel(n_input=11, n_output=2)
  # Optimizer는 확률적 경사하강법 알고리즘인 SGD를 사용함
  # 경사하강법 알고리즘은 파라미터(weight, bias)를 업데이트하므로 첫번째 인자로 파라미터들을 넣어줌.
  # 두번째 인자로는 훈련할 때 업데이트할 보폭(크기)을 정하는 learning rate를 지정해줌.
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

  return my_model, optimizer


# 훈련 함수
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  # wandb.config.epochs = 1000
  n_epochs = wandb.config.epochs
  # Loss function으로 가장 많이 사용하는 MSELoss를 사용함
  loss_fn = nn.CrossEntropyLoss()  # Use a built-in loss function
  next_print_epoch = 100

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      # === forward ===
      output_train = model(train_batch['input'])
      # target: 정답 데이터
      # target batch shape (16) -> (16,1)로 맞춰줌
      loss = loss_fn(output_train, train_batch['target'])
      # 손실값 얻어서 loss_train에 더해줌
      loss_train += loss.item()
      num_trains += 1
      # ===============

      # gradient=0으로 해줌으로써 이전 gradient가 다음 backward에 영향을 미치지 않게 함
      optimizer.zero_grad()
      # backward
      loss.backward()
      # weight에서 lr * gradient를 빼줌으로써 weight(가중치) 업데이트
      optimizer.step()


    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        output_validation = model(validation_batch['input'])
        # target batch shape (16) -> (16,1)로 맞춰줌
        loss = loss_fn(output_validation, validation_batch['target'])
        loss_validation += loss.item()
        num_validations += 1

    # wandb.log로 학습에 대한 정보를 wandb로 보내면 그래프와 함께 출력된 로그를 확인할 수 있음.
    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations
    })

    # Epoch 100 단위로 출력
    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 100


def main(args):
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [20, 20],
  }

  # wandb 초기화
  wandb.init(
    mode="online" if args.wandb else "disabled",
    project="my_model_training",
    notes="My first wandb experiment",
    tags=["my_model", "titanic_dataset"],
    name=current_time_str,
    config=config
  )
  print(args)
  print(wandb.config)

  # train, validation dataset 획득
  train_data_loader, validation_data_loader = get_data()

  # model, optimizer 획득
  linear_model, optimizer = get_model_and_optimizer()

  # gradient 값을 시각해주기 위해 model을 인자로 넘김
  wandb.watch(linear_model)

  print("#" * 50, 1)

  # 훈련 시작
  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader
  )
  #test(linear_model, test_data_loader)
  wandb.finish()


# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":
  parser = argparse.ArgumentParser()

  parser.add_argument(
    "--wandb", action=argparse.BooleanOptionalAction, default=False, help="True or False"
  )

  parser.add_argument(
    "-b", "--batch_size", type=int, default=512, help="Batch size (int, default: 512)"
  )

  parser.add_argument(
    "-e", "--epochs", type=int, default=1_000, help="Number of training epochs (int, default:1_000)"
  )

  args = parser.parse_args("")

  main(args)



BASE PATH /Users/jaewoogwak/git/link_dl
Namespace(wandb=False, batch_size=512, epochs=1000)
{'epochs': 1000, 'batch_size': 512, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20]}
getgetet
wandb.confi.hidden unit list [20, 20]
################################################## 1
Epoch 100, Training loss 0.6112, Validation loss 0.6093
Epoch 200, Training loss 0.6092, Validation loss 0.5882
Epoch 300, Training loss 0.5985, Validation loss 0.5806
Epoch 400, Training loss 0.5885, Validation loss 0.5765
Epoch 500, Training loss 0.5894, Validation loss 0.5778
Epoch 600, Training loss 0.5820, Validation loss 0.5804
Epoch 700, Training loss 0.5967, Validation loss 0.5745
Epoch 800, Training loss 0.5804, Validation loss 0.5767
Epoch 900, Training loss 0.5770, Validation loss 0.5781
Epoch 1000, Training loss 0.5598, Validation loss 0.5713


- titanic_dataset 코드를 f번 코드에 적용
- Model의 input, output을 변경해주었고, 그에 맞게 batch값의 shape 변환을 적절히 해줌

### Wandb에서 확인한 훈련 과정 데이터

https://wandb.ai/jaewoogwak/my_model_training/runs/j0cddod1/overview?workspace=user-jaewoo010207

![wandb_image](https://raw.githubusercontent.com/jaewoogwak/Deep_learning/master/homework/hw2/images/wandb.png)

| Epoch         | Training Loss | Validation Loss |
|---------------|---------------|-----------------|
| 10,000        | 0.4305        | 0.4319          |



### Activation function 채용 과정
같은 조건에서 매 훈련마다 training loss & validation loss가 조금씩 다르게 나왔다. 그래서 activation function마다 5번씩 같은 조건으로 훈련하여 얻은 training loss & validation loss를 모두 합하고 5로 나눈 값으로 평균 training loss & validation loss을 얻었다. 이 값을 비교하여 **가장 낮은 validation loss를 보이는 activation function을 사용**하였다.

- ATL = Average Training Loss
- AVL = Average Validation Loss

|     |       | Training Loss | Validation Loss |
|-----|-------|---------------|-----------------|
| ReLU | Epoch 1000 | 0.6030 | 0.5555 |
|      | Epoch 1000 | 0.5758 | 0.5701 |
|      | Epoch 1000 | 0.5610 | 0.5556 |
|      | Epoch 1000 | 0.5743 | 0.5455 |
|      | Epoch 1000 | 0.5649 | 0.5962 |
|      | ATL   | 0.5758 | -     |
|      | AVL   | 0.5646 | -     |
| LeakyReLU | Epoch 1000 | 0.5490 | 0.6259 |
|           | Epoch 1000 | 0.5515 | 0.5724 |
|           | Epoch 1000 | 0.5782 | 0.5924 |
|           | Epoch 1000 | 0.5601 | 0.5827 |
|           | Epoch 1000 | 0.5767 | 0.5394 |
|           | ATL   | 0.5631 | -     |
|           | AVL   | 0.58256 | -     |
| ELU | Epoch 1000 | 0.5500 | 0.6058 |
|     | Epoch 1000 | 0.5606 | 0.5686 |
|     | Epoch 1000 | 0.5523 | 0.5518 |
|     | Epoch 1000 | 0.5488 | 0.6568 |
|     | Epoch 1000 | 0.5633 | 0.5623 |
|     | ATL   | 0.555  | -     |
|     | AVL   | 0.58906 | -     |
| PReLU | Epoch 1000 | 0.5549 | 0.5922 |
|      | Epoch 1000 | 0.5899 | 0.5685 |
|      | Epoch 1000 | 0.5918 | 0.5485 |
|      | Epoch 1000 | 0.5653 | 0.6263 |
|      | Epoch 1000 | 0.5777 | 0.5911 |
|      | ATL   | 0.57592 | -     |
|      | AVL   | 0.58532 | -     |


**가장 낮은 평균 validation loss를 보인 Activation function은 ReLU이다.**

### [고찰] 왜 같은 조건에서 훈련하는데도 매번 손실값이 다르게 나올까?
훈련할 때마다 결과로 training loss와 validation loss가 매번 조금씩 다르게 나왔다.

이 부분을 처음에 이해하기가 어려웠다.

분명 train.csv는 그대로인데 말이다.

그래서 전처리하는 부분부터 천천히 다시 봤고 그 결과, Random split 때문임을 깨달았다.

아래 데이터셋을 랜덤 스플릿하여 train dataset과 validation dataset으로 나누는 것이 원인이었다.

```python
dataset = TitanicDataset(train_X.values, train_y.values)

train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
```

`dataset`까지는 데이터에 전혀 문제 없다. (전처리과정을 거치긴하지만)

그런데 `dataset`을 `random_split()` 함수로 8:2 비율로 나눠줌으로써 `train_dataset`과 `validation_dataset`은 항상 다른 배치 순서를 가지는 데이터셋이 된다.

그러니 훈련할 때도 입력값이 매번 다른 순서로 입력될 것이다.

결국 매 훈련마다 training loss와 validation loss가 다른 것은 당연한 것이다.


```

### [고찰] Overfitting 문제
Traning loss가 매우 낮은데 validation loss가 높다면 Overfitting(모델이 훈련 데이터에만 지나치게 잘 맞고 일반적인 데이터를 제대로 수행하지 못함) 신호일 수 있다.

실제로 Epoch를 엄청나게 늘려도 어느 순간부터는 training loss와 validation loss의 차이가 점점 벌어지기 시작한다.



결국 Training loss & Validation loss 그래프 모두 수렴해서 안정화되는 것이 중요하다. 이러한 모델을 목표로 해야한다. 훈련 값을 잘 학습하면서 새로운 데이터에 대해서도 잘 적응하는 모델을 만들어야한다.


# [요구사항3] 테스트 및 submission.csv 생성

## Activation function: ReLU

위의 Activation function 채용 과정에서 가장 좋은 성능을 보이는 Activation function은 ReLU로 판단하였고 ReLU로 모델을 구성하였다.

In [17]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse

from torch.utils.data import Dataset, DataLoader, random_split

from pathlib import Path
BASE_PATH = str(Path(os.path.abspath('submission.py')).resolve().parent.parent.parent.parent)


import csv

import sys
sys.path.append(BASE_PATH)

print("BASE PATH", BASE_PATH)

from _03_your_code.homework.hw2.titanic_dataset import get_preprocessed_dataset



# Train, Validation dataset을 반환하는 함수
def get_data():
  # titanic_dataset.py에서 사용했던 get_preprocessed_dataset 함수를 통해 전처리된 데이터 얻기
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
  # 해당 데이터를 Dataloader에 넣어 훈련에 사용할 수 있는 dataloader 얻기
  # batch size는 wandb.default = 512임.
  # validation dataset size = validation data loader's batch size = 178로 validation dataset을 한번에 훈련시킴
  train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  return train_data_loader, validation_data_loader, test_data_loader


class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    print("wandb.confi.hidden unit list", wandb.config.n_hidden_unit_list)

    # nn.Sequential은 인자로 들어온 모듈들에 입력값을 순서대로 적용해준다.
    # Input layer - hidden layer(1) - Output layer 구조로 이루어짐
    # (input, 20) (20, 20) (20, output)
    # Activation function: ELU
    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      nn.ELU(),
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      nn.ELU(),
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x


# Model & Optimizer를 얻는 함수
def get_model_and_optimizer():
  # titanic dataset은 feature가 11개이고 output(생존여부)이 생존,사망
  my_model = MyModel(n_input=11, n_output=2)
  # Optimizer는 확률적 경사하강법 알고리즘인 SGD를 사용함
  # 경사하강법 알고리즘은 파라미터(weight, bias)를 업데이트하므로 첫번째 인자로 파라미터들을 넣어줌.
  # 두번째 인자로는 훈련할 때 업데이트할 보폭(크기)을 정하는 learning rate를 지정해줌.
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

  return my_model, optimizer


# 훈련 함수
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
  # wandb.config.epochs = 1000
  n_epochs = wandb.config.epochs
  # Loss function으로 가장 많이 사용하는 MSELoss를 사용함
  loss_fn = nn.CrossEntropyLoss()
  next_print_epoch = 1000

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      # === forward ===
      output_train = model(train_batch['input'])
      # target: 정답 데이터
      loss = loss_fn(output_train, train_batch['target'])
      # 손실값 얻어서 loss_train에 더해줌
      loss_train += loss.item()
      num_trains += 1
      # ===============

      # gradient=0으로 해줌으로써 이전 gradient가 다음 backward에 영향을 미치지 않게 함
      optimizer.zero_grad()
      # backward
      loss.backward()
      # weight에서 lr * gradient를 빼줌으로써 weight(가중치) 업데이트
      optimizer.step()


    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        output_validation = model(validation_batch['input'])
        loss = loss_fn(output_validation, validation_batch['target'])
        loss_validation += loss.item()
        num_validations += 1


    # wandb.log로 학습에 대한 정보를 wandb로 보내면 그래프와 함께 출력된 로그를 확인할 수 있음.
    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations,
    })

    # Epoch 1000 단위로 출력
    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 1000


def test(my_model, test_data_loader):
  print("[TEST]")
  # test data loader로부터 batch를 차례차례 가져옴
  batch = next(iter(test_data_loader))
  print("{0}".format(batch['input'].shape))
  # 모델에 input을 넣어 output을 얻음 output_batch.shape = (418,2)
  output_batch = my_model(batch['input'])

  prediction_batch = torch.argmax(output_batch, dim=1)
  result = []

  for idx, prediction in enumerate(prediction_batch, start=892):
    result.append((idx, prediction.item()))

  return result



def main(args):
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [20, 20],
  }

  # wandb 초기화
  wandb.init(
    mode="online" if args.wandb else "disabled",
    project="my_model_training",
    notes="My first wandb experiment",
    tags=["my_model", "titanic_dataset"],
    name=current_time_str,
    config=config
  )
  print(args)
  print(wandb.config)

  # train, validation dataset 획득
  train_data_loader, validation_data_loader, test_data_loader = get_data()

  # model, optimizer 획득
  linear_model, optimizer = get_model_and_optimizer()

  # gradient 값을 시각해주기 위해 model을 인자로 넘김
  wandb.watch(linear_model)

  print("#" * 50, 1)

  # 훈련 시작
  # linear_model을 훈련시킴
  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader,
  )

  # test dataset 검증
  # 훈련된 linear_model에 test_dataset을 적용시켜봄.
  result = test(linear_model, test_data_loader)

  
  # CSV 파일 생성  
  csv_file = "submission.csv"
  
  with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(["PassengerId", "Survived"])

    for key, value in result:
      writer.writerow([key, value])

  print(f"Data has been written to {csv_file}")
  wandb.finish()


# https://docs.wandb.ai/guides/track/config
if __name__ == "__main__":
  parser = argparse.ArgumentParser()

  parser.add_argument(
    "--wandb", action=argparse.BooleanOptionalAction, default=False, help="True or False"
  )

  parser.add_argument(
    "-b", "--batch_size", type=int, default=512, help="Batch size (int, default: 512)"
  )

  parser.add_argument(
    "-e", "--epochs", type=int, default=10_000, help="Number of training epochs (int, default:1_000)"
  )

  args = parser.parse_args("")

  main(args)



BASE PATH /Users/jaewoogwak/git/link_dl
Namespace(wandb=False, batch_size=512, epochs=10000)
{'epochs': 10000, 'batch_size': 512, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20]}
getgetet
wandb.confi.hidden unit list [20, 20]
################################################## 1
Epoch 1000, Training loss 0.5711, Validation loss 0.5771
Epoch 2000, Training loss 0.5568, Validation loss 0.5609
Epoch 3000, Training loss 0.5434, Validation loss 0.5504
Epoch 4000, Training loss 0.5389, Validation loss 0.5381
Epoch 5000, Training loss 0.5314, Validation loss 0.5243
Epoch 6000, Training loss 0.5192, Validation loss 0.5096
Epoch 7000, Training loss 0.4877, Validation loss 0.4929
Epoch 8000, Training loss 0.4731, Validation loss 0.4759
Epoch 9000, Training loss 0.4611, Validation loss 0.4598
Epoch 10000, Training loss 0.4461, Validation loss 0.4468
[TEST]
torch.Size([418, 11])
Data has been written to submission.csv


In [21]:
submission_csv = pd.read_csv("./submission.csv")

In [22]:
print(submission_csv)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
5            897         0
6            898         1
7            899         0
8            900         1
9            901         0
10           902         0
11           903         0
12           904         1
13           905         0
14           906         1
15           907         1
16           908         0
17           909         0
18           910         0
19           911         0
20           912         0
21           913         0
22           914         1
23           915         1
24           916         1
25           917         0
26           918         1
27           919         0
28           920         0
29           921         0
30           922         0
31           923         0
32           924         0
33           925         0
34           926         1
35           927         0
3

In [23]:
print(submission_csv.shape)

(418, 2)


### [고찰] 훈련과정 중 어느 Epoch 시점에 테스트를 수행하여 submission.csv를 구성해야 하는지 고찰하기

이번 과제를 통해 수십번의 훈련을 하며 얻는 결론은 "정답은 없다." 이다.

내가 선택한 기준은 아래와 같다. 

> training loss가 낮으면서 validation loss와 큰 차이가 없어야 한다

1만번의 epoch를 기준으로 어느 시점에서 가장 낮은 loss 값을 보이는지 찾아보았는데,

어떤 케이스에서는 2천번대의 초반 시점에서 위의 조건에 가장 부합했는데

또 어떤 케이스는 최종 epoch에서 위의 조건에 가장 부합했다.

매 실행마다 랜덤 스플릿에 의해 데이터셋 배치도 달라지니 어느 특정 epoch에서 가장 좋은 결과를 보인다고 말하는건 쉽지 않다고 생각했다.

테스트 결과인 submission.csv를 캐글에 제출하면 캐글 스코어도 매번 다르게 나왔다.

그래서 나는 설정한 epoch 만큼 훈련이 끝난 뒤 테스트를 수행하기로 결정하였다.

그래서 아래와 같이 `training_loop`를 마친 후에 `test`를 하게된다.

```python
# 훈련 시작
  # linear_model을 훈련시킴
  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader,
  )

  # test dataset 검증
  # 훈련된 linear_model에 test_dataset을 적용시켜봄.
  result = test(linear_model, test_data_loader)

```

### [고찰] 삽질의 연속
이번 과제는 시작부터 정말 쉽지 않았다. 다른 파일에서 함수를 임포트하려는데 경로 문제때문에 에러가 발생했다. 다행히 나와 같은 사람이 많았고 stackoverflow를 보며 해결할 수 있었다. 
이후 submission.csv를 제출하면서 이상함을 느꼈다. 생각보다 캐글 스코어가 너무 낮게 나오는 것이었다. 38점, 42점.. 형편없는 점수였다. 
문제를 찾기 위해 코드를 처음부터 다시 읽어보고, 그러면서 모르는 부분이 있으면 또 공부해보고 하는 과정의 연속이었다. 

그렇게 문제의 원인을 발견할 수 있었다. 
training_loop 함수를 통해 모델을 훈련시켜놓고 test 함수에 해당 모델을 인자로 넘겨주지 않은 것이었다. 참 어처구니 없는 상황이었다. 
마치 내가 딥러닝 수업을 듣고 열심히 공부해놓고 딥러닝 수업을 듣지 않는 내 친구가 시험을 보러 간 꼴이었다. 당연히 그 친구는 시험을 잘 볼리 없다. 
이런 실수는 FCN에 대한 전반적인 이해가 부족했기 때문이라고 생각했다. 모델을 훈련하고 테스트 데이터셋에 해당 모델을 적용해보는 이러한 과정에 대한 이해가 부족했었다. 
그러나 이런 삽질을 바탕으로 앞으로의 나는 이런 문제는 쉽게 해결할 수 있게 되었다. 값진 경험이라고 생각한다.



### [고찰] 실습을 통해 이론을 이해하게 되다
처음에는 MSELoss 손실함수를 사용했었다. 그러다가 CrossEntropyLoss 손실함수를 사용해봤다. 사실 처음에는 뭣도 모르고 그냥 손실함수를 바꿔본 것이었다. 그런데 좋은 결과를 냈었고, Classification(분류)에는 CrossEntropyLoss를 사용하고 Regression(회귀)에는 MSELoss를 대표적으로 사용하는걸 알게 되었다. 나는 타이타닉 예제는 주어진 데이터를 바탕으로 생존과 사망을 분류하는 문제라고 생각했다. 또한 Regression은 시간이 흐름에 따라 각 데이터가 존재하며 이전 데이터를 바탕으로 예측하는 것으로 알고 있는데, 이것은 타이타닉 예제와는 다르다고 생각했다. 

위와 같이 이번 과제는 코드를 바탕으로 먼저 시도해보고 코드를 이해하면서 궁금한 부분을 떼어내서 찾아보는 식으로 공부했다. 그러다보니 배운 내용을 적용할 수 있게 되었고 실습을 통해 이론이  사실임을 깨달았을 때는 굉장히 기분이 좋았다.

# [요구사항4] submission.csv 제출 및 등수 확인

![img](https://raw.githubusercontent.com/jaewoogwak/Deep_learning/master/homework/hw2/images/kaggle_rank.png)

# [요구사항5] Wandb 페이지 생성 및 URL 제출

### My Wandb Page 🔽

https://wandb.ai/jaewoogwak/my_model_training/runs/j0cddod1/overview?workspace=user-jaewoo010207

# [숙제 후기]
이번 과제2는 학교를 다니면서 내가 만났던 과제 중에 손에 꼽을만큼 힘들었던 과제였다.

그치만 위에서 고찰이 넘쳐나는만큼 얻어가는게 너무나도 많아서 좋다.

또한 교수님께서 개발자스러운 면이 있으시다고 생각한다. (너무 좋다)

다른 교수님들은 옛날 스타일에 그대로 고착된 느낌인데

한연희 교수님은 주피터 노트북에 과제를 제출하라고 하고 (다른 교수님이었으면 한글 문서에 제출하라고 했을 거 같다)

깃헙에 코드도 다 올려 놓으시고 계속해서 과제 업데이트하고 git으로 쉽게 관리할 수 있게 구성해놓으셨다. (다른 교수님이면 그냥 EL에 코드 압축해서 올려놓으실 거 같다)

또한 캐글을 써보는 과제는 정말 최신 트렌드를 따라간 거 같다.

학부생 수준에서 캐글을 써보다니 정말 귀중한 경험이다.

이제 7주차가 끝난 시점임에도 감히 내가 지금까지 들었던 수업 중 최고로 뽑을 수 있을 거 같다.

시험을 잘 봐서 성적이 잘 나오면 한기대 최고의 강의로 영구결번 할 수 있을 거 같다.

보통 과제를 하면서 공부가 되는 수업은 찾아보기 힘들다.

과제를 하면서 공부할 수 있는게 어쩌면 당연한 것일 수도 있지만 그렇게 수업을 설계하기 힘들다.

그러나 한연희 교수님의 과제를 해결하다보면 수업시간에 배운 이론을 적용하게 되고 이해하며 즐거움을 느끼는 순간을 경험할 수 있다.

이번 과제2는 고통스럽지만 좋았다.

그런데 과제 양이 너무 많은 거 같습니다 교수님..